In [ ]:
import h5py
import numpy as np

# Load dataset
train_data = h5py.File("/Users/praveenkumardevamane/Downloads/archive (2)/train_catvsnoncat.h5", "r")
test_data = h5py.File("/Users/praveenkumardevamane/Downloads/archive (2)/test_catvsnoncat.h5", "r")

# Extract training and testing data
X_train = train_data["train_set_x"][:]      # shape: (m_train, 64, 64, 3)
Y_train = train_data["train_set_y"][:]      # shape: (m_train,)

X_test = test_data["test_set_x"][:]         # shape: (m_test, 64, 64, 3)
Y_test = test_data["test_set_y"][:]    

In [ ]:
X_train = X_train.reshape(X_train.shape[0], -1).T / 255.0   # (12288, m_train)
X_test = X_test.reshape(X_test.shape[0], -1).T / 255.0      # (12288, m_test)

Y_train = Y_train.reshape(1, Y_train.shape[0])  # (1, m_train) - FIXED
Y_test = Y_test.reshape(1, Y_test.shape[0])  

print("\nAfter reshaping:")
print(f"X_train: {X_train.shape}")
print(f"Y_train: {Y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"Y_test: {Y_test.shape}")

In [ ]:
def sigmoid(z):
    """Sigmoid activation function with numerical stability"""
    # Clip z to prevent overflow
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

In [ ]:

num_features = X_train.shape[0]  
m2 = X_train.shape[1]             


w2 = np.zeros((num_features, 1))  
b2 = 0.0


alpha2 = 0.001


dw2 = np.zeros_like(w2)
db2 = 0.0

In [ ]:
def compute(X, Y, w, b, alpha=0.001, iterations=1000):
    """
    Performs gradient descent for logistic regression
    
    Arguments:
    X -- input data, shape (n_features, m_examples)
    Y -- true labels, shape (1, m_examples)
    w -- weights, shape (n_features, 1)
    b -- bias, scalar
    alpha -- learning rate
    iterations -- number of iterations
    
    Returns:
    w -- optimized weights
    b -- optimized bias
    cost -- final cost
    """
    m = X.shape[1]
    costs = []

    for i in range(iterations):
        # Forward propagation
        z = np.dot(w.T, X) + b           # (1, m)
        A = sigmoid(z)                   # (1, m)
        
        # Clip A to prevent log(0) errors
        A = np.clip(A, 1e-15, 1 - 1e-15)
        
        # Cost computation
        cost = -(1/m) * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))
        
        # Backward propagation
        dz = A - Y                       # (1, m)
        dw = (1/m) * np.dot(X, dz.T)     # (n_features, 1)
        db = (1/m) * np.sum(dz)          # scalar
        
        # Update parameters
        w = w - alpha * dw
        b = b - alpha * db
        
        # Store cost every 100 iterations
        if i % 100 == 0:
            costs.append(cost)
            print(f"Cost after iteration {i}: {cost}")

    return w, b, cost, costs

In [ ]:
def predict(X, w, b):
    """
    Make predictions using learned parameters
    
    Arguments:
    X -- input data, shape (n_features, m_examples)
    w -- weights, shape (n_features, 1)
    b -- bias, scalar
    
    Returns:
    predictions -- binary predictions, shape (1, m_examples)
    """
    z = np.dot(w.T, X) + b
    A = sigmoid(z)
    predictions = (A > 0.5).astype(int)
    return predictions

# Initialize parameters
num_features = X_train.shape[0]  # 12288
m_train = X_train.shape[1]       # number of training examples

w = np.random.randn(num_features, 1) * 0.01  # Small random initialization
b = 0.0

# Train the model
print("\nTraining the model...")
w_trained, b_trained, final_cost, cost_history = compute(X_train, Y_train, w, b, alpha=0.005, iterations=2000)

# Make predictions
train_predictions = predict(X_train, w_trained, b_trained)
test_predictions = predict(X_test, w_trained, b_trained)

# Calculate accuracy
train_accuracy = 100 - np.mean(np.abs(train_predictions - Y_train)) * 100
test_accuracy = 100 - np.mean(np.abs(test_predictions - Y_test)) * 100

print(f"\nFinal Results:")
print(f"Train Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Final Cost: {final_cost:.6f}")


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

def preprocess_image(image_path, target_size=(64, 64)):
    """
    Preprocess a single image for prediction
    
    Arguments:
    image_path -- path to the image file
    target_size -- tuple (height, width) to resize the image
    
    Returns:
    image_vector -- preprocessed image as a column vector, shape (12288, 1)
    """
    try:
        # Load and resize image
        image = Image.open(image_path)
        
        # Convert to RGB if not already (handles RGBA, grayscale, etc.)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # Resize to 64x64 pixels
        image = image.resize(target_size)
        
        # Convert to numpy array
        image_array = np.array(image)
        
        # Reshape to column vector and normalize
        image_vector = image_array.reshape(-1, 1) / 255.0
        
        return image_vector, image_array
    
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None, None

def predict_single_image(image_path, w, b, show_image=True):
    """
    Predict whether a single image contains a cat or not
    
    Arguments:
    image_path -- path to the image file
    w -- trained weights
    b -- trained bias
    show_image -- whether to display the image
    
    Returns:
    prediction -- 1 if cat, 0 if not cat
    probability -- probability of being a cat
    """
    # Preprocess the image
    image_vector, image_array = preprocess_image(image_path)
    
    if image_vector is None:
        return None, None
    
    # Make prediction
    z = np.dot(w.T, image_vector) + b
    probability = sigmoid(z)[0, 0]  # Extract scalar value
    prediction = 1 if probability > 0.5 else 0
    
    # Display results
    if show_image:
        plt.figure(figsize=(8, 4))
        
        # Show the image
        plt.subplot(1, 2, 1)
        plt.imshow(image_array)
        plt.title(f"Input Image\n{os.path.basename(image_path)}")
        plt.axis('off')
        
        # Show prediction
        plt.subplot(1, 2, 2)
        result_text = "CAT" if prediction == 1 else "NOT CAT"
        color = 'green' if prediction == 1 else 'red'
        
        plt.text(0.5, 0.6, result_text, fontsize=20, fontweight='bold', 
                ha='center', va='center', color=color)
        plt.text(0.5, 0.4, f"Probability: {probability:.3f}", fontsize=14, 
                ha='center', va='center')
        plt.text(0.5, 0.2, f"Confidence: {abs(probability-0.5)*2:.1%}", fontsize=12, 
                ha='center', va='center')
        
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.axis('off')
        plt.title("Prediction Result")
        
        plt.tight_layout()
        plt.show()
    
    return prediction, probability

In [ ]:
# Test one image and see the result
prediction, probability = predict_single_image('/Users/praveenkumardevamane/Downloads/cats/Data/cat_1.png', w_trained, b_trained)
print(f"Prediction: {'Cat' if prediction == 1 else 'Not Cat'}")
print(f"Probability: {probability:.3f}")

In [ ]:
def test_with_webcam():
    """
    Function to capture image from webcam and test
    Requires cv2 (opencv-python)
    """
    try:
        import cv2
        
        # Initialize webcam
        cap = cv2.VideoCapture(0)
        
        print("Press SPACE to capture image, ESC to exit")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            cv2.imshow('Webcam - Press SPACE to capture', frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord(' '):  # Space key
                # Save captured frame
                cv2.imwrite('captured_image.jpg', frame)
                cap.release()
                cv2.destroyAllWindows()
                
                # Test the captured image
                prediction, prob = predict_single_image('captured_image.jpg', w_trained, b_trained)
                return prediction, prob
                
            elif key == 27:  # ESC key
                break
        
        cap.release()
        cv2.destroyAllWindows()
        
    except ImportError:
        print("OpenCV not installed. Install with: pip install opencv-python")

In [ ]:
import cv2
import numpy as np

def real_time_cat_detector(w, b):
    """
    Captures video from the webcam and performs real-time cat detection.
    
    Arguments:
    w -- trained weights from the model
    b -- trained bias from the model
    """
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
        
    print("Starting real-time detector. Press 'ESC' to exit.")

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break
            
        # --- 1. Preprocess the Frame ---
        # Convert frame from BGR (OpenCV default) to RGB (model was trained on RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Resize to the model's expected input size (64x64)
        resized_frame = cv2.resize(rgb_frame, (64, 64))
        
        # Reshape to a column vector and normalize
        image_vector = resized_frame.reshape(-1, 1) / 255.0
        
        # --- 2. Make a Prediction ---
        z = np.dot(w.T, image_vector) + b
        probability = sigmoid(z)[0, 0]
        prediction = 1 if probability > 0.5 else 0
        
        # --- 3. Display the Result on the Frame ---
        if prediction == 1:
            result_text = f"Cat ({probability:.2f})"
            color = (0, 255, 0) # Green in BGR
        else:
            result_text = f"Not Cat ({probability:.2f})"
            color = (0, 0, 255) # Red in BGR
        
        # Put the prediction text on the original frame
        cv2.putText(frame, result_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        
        # Display the resulting frame
        cv2.imshow('Real-time Cat Detector', frame)
        
        # --- 4. Handle Exit ---
        # Press 'ESC' (ASCII value 27) to exit the loop
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # Release the capture and destroy all windows
    cap.release()
    cv2.destroyAllWindows()

In [9]:
# Capture and test image from webcam
prediction, probability = real_time_cat_detector(w, b)